In [1]:
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';')
total_records = len(df)
total_columns = len(df.columns)
print(f"Total Records: {total_records}")
print(f"Total Columns: {total_columns}")

# Expected domains
expected_domains = {
    
    'age': (18, 95),           # Based on your data max
    'balance': (-8019, 102127),# Accepting current observed range
    'day': (1, 31),            # Days of a month, already fine
    'duration': (0, 5000),     # max is 4918, round to 5000
    'campaign': (1, 100),      # Current max is 63, but keeping 100 is realistic
    'pdays': (-1, 999),        # max is 871, -1 is valid, 999 is fine as upper limit
    'previous': (0, 300),       # max is 275, rounding to 300 for flexibility
    

    'job': ["admin.", "unknown", "unemployed", "management", "housemaid",
            "entrepreneur", "student", "blue-collar", "self-employed",
            "retired", "technician", "services"],

    'marital': ["married", "divorced", "single"],
    'education': ["unknown", "secondary", "primary", "tertiary"],
    'default': ["yes", "no"],
    'housing': ["yes", "no"],
    'loan': ["yes", "no"],
    'contact': ["unknown", "telephone", "cellular"],
    'day': (1, 31),
    'month': ["jan", "feb", "mar", "apr", "may", "jun",
              "jul", "aug", "sep", "oct", "nov", "dec"],
    'poutcome': ["unknown", "other", "failure", "success"],
    'y': ["yes", "no"]
}

# Prepare Report List
report = []

def add_issue(pillar, column, description, rows):
    count = len(rows) if rows else 0
    percentage = (count / total_records) * 100
    percentage_str = f"{percentage:.2f}%"
    row_list = rows if rows else None
    report.append({
        'Pillar': pillar,
        'Column Name': column,
        'Issue description': description,
        'Row number': row_list,
        'Issue Record Count': count,
        'Percentage': percentage_str
    })

# ----------------------------
# Completeness
missing_cols = df.columns[df.isnull().any()].tolist()
if missing_cols:
    rows = df[df[missing_cols].isnull().any(axis=1)].index.tolist()
    add_issue('Completeness', ', '.join(missing_cols), 'Missing values', rows)
else:
    add_issue('Completeness', 'All Columns', 'No missing values found', [])

# ----------------------------
# Uniqueness
duplicates = df[df.duplicated()].index.tolist()
add_issue('Uniqueness', 'All Columns', 'Duplicate records', duplicates)

# ----------------------------
# Accuracy (Numeric vs Categorical)
accuracy_found = False

for col, valid_values in expected_domains.items():
    if col in df.columns:
        if isinstance(valid_values, tuple):
            min_val, max_val = valid_values
            invalid_rows = df[~df[col].between(min_val, max_val)].index.tolist()
            if invalid_rows:
                add_issue('Accuracy', col, f'Values out of range {min_val} - {max_val}', invalid_rows)
                accuracy_found = True
        else:
            invalid_rows = df[~df[col].isin(valid_values)].index.tolist()
            if invalid_rows:
                add_issue('Accuracy', col, 'Values not in expected domain', invalid_rows)
                accuracy_found = True

if not accuracy_found:
    add_issue('Accuracy', 'All relevant columns', 'No accuracy issues found', [])

# ----------------------------
# Consistency (Example rule for day)
if 'day' in df.columns:
    invalid_days = df[~df['day'].between(1, 31)].index.tolist()
    add_issue('Consistency', 'day', 'Values out of range 1-31', invalid_days)
else:
    add_issue('Consistency', 'Relevant Columns', 'No consistency rule applied', [])

# ----------------------------
# Timeliness (No dates in data)
add_issue('Timeliness', 'Date Fields', 'No timeliness data provided for check', [])

# ----------------------------
# Integrity (No defined integrity constraints in provided data)
add_issue('Integrity', 'All Columns', 'No integrity constraints defined', [])

# ----------------------------
# Final Report
dq_report = pd.DataFrame(report)
dq_report = dq_report.sort_values(by='Pillar').reset_index(drop=True)

from IPython.display import display
display(dq_report.style.set_properties(**{'text-align': 'left'})
        .set_caption("Final Data Quality Report - All 6 Pillars (with Totals)"))

# Optional CSV export
# dq_report.to_csv('final_data_quality_report_with_totals.csv', index=False)


Total Records: 45211
Total Columns: 17


,Pillar,Column Name,Issue description,Row number,Issue Record Count,Percentage
0,Accuracy,All relevant columns,No accuracy issues found,None,0,0.00%
1,Completeness,All Columns,No missing values found,None,0,0.00%
2,Consistency,day,Values out of range 1-31,None,0,0.00%
3,Integrity,All Columns,No integrity constraints defined,None,0,0.00%
4,Timeliness,Date Fields,No timeliness data provided for check,None,0,0.00%
5,Uniqueness,All Columns,Duplicate records,None,0,0.00%


In [2]:
import pandas as pd

# Select numeric columns only
numeric_cols = df.select_dtypes(include='number')

# Calculate min, max, mean
summary_stats = pd.DataFrame({
    'Min': numeric_cols.min(),
    'Max': numeric_cols.max(),
    'Mean': numeric_cols.mean()
})

# Display
print(summary_stats)


           Min     Max         Mean
age         18      95    40.936210
balance  -8019  102127  1362.272058
day          1      31    15.806419
duration     0    4918   258.163080
campaign     1      63     2.763841
pdays       -1     871    40.197828
previous     0     275     0.580323


In [3]:
import pandas as pd
import numpy as np
import re

# --------------------------
# Load Data (Suppress Dtype Warning)
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';', dtype=str)
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';')
total_records = len(df)
total_columns = len(df.columns)
print(f"Total Records: {total_records}")
print(f"Total Columns: {total_columns}")

# --------------------------
# Expected Domains
expected_domains = {
    'age': (18, 95),
    'balance': (-8019, 102127),
    'day': (1, 31),
    'duration': (0, 5000),
    'campaign': (1, 100),
    'pdays': (-1, 999),
    'previous': (0, 300),

    'job': ["admin.", "unknown", "unemployed", "management", "housemaid",
            "entrepreneur", "student", "blue-collar", "self-employed",
            "retired", "technician", "services"],
    'marital': ["married", "divorced", "single"],
    'education': ["unknown", "secondary", "primary", "tertiary"],
    'default': ["yes", "no"],
    'housing': ["yes", "no"],
    'loan': ["yes", "no"],
    'contact': ["unknown", "telephone", "cellular"],
    'month': ["jan", "feb", "mar", "apr", "may", "jun",
              "jul", "aug", "sep", "oct", "nov", "dec"],
    'poutcome': ["unknown", "other", "failure", "success"],
    'y': ["yes", "no"]
}

categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# --------------------------
# Reporting Utility
report = []

def add_issue(pillar, column, description, rows):
    count = len(rows) if rows else 0
    percentage = (count / total_records) * 100
    if 0 < percentage < 0.01:
        percentage_str = "<0.01%"
    else:
        percentage_str = f"{percentage:.2f}%"
    row_list = rows if rows else None
    report.append({
        'Pillar': pillar,
        'Column Name': column,
        'Issue description': description,
        'Row number': row_list,
        'Issue Record Count': count,
        'Percentage': percentage_str
    })


# --------------------------
# 1 Completeness Check
missing_cols = df.columns[df.isnull().any()].tolist()
if missing_cols:
    rows = df[df[missing_cols].isnull().any(axis=1)].index.tolist()
    add_issue('Completeness', ', '.join(missing_cols), 'Missing values', rows)
else:
    add_issue('Completeness', 'All Columns', 'No missing values found', [])


# --------------------------
# 2 Uniqueness Check
duplicates = df[df.duplicated()].index.tolist()
add_issue('Uniqueness', 'All Columns', 'Duplicate records', duplicates)


# --------------------------
# 3 Accuracy Checks

accuracy_found = False

# --- Data Type Validation & Numeric Range Checks (Fixed!)
for col in numeric_columns:
    if col in df.columns:
        # Convert to numeric safely
        converted_col = pd.to_numeric(df[col], errors='coerce')

        # Identify rows where conversion failed (non-numeric values)
        non_numeric_rows = df[pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()].index.tolist()
        if non_numeric_rows:
            add_issue('Accuracy', col, 'Non-numeric value found in numeric column', non_numeric_rows)
            accuracy_found = True

        # After conversion, check valid ranges only for numeric
        valid_mask = converted_col.between(*expected_domains[col])
        invalid_range_rows = df[~valid_mask & converted_col.notna()].index.tolist()
        if invalid_range_rows:
            add_issue('Accuracy', col, f'Values out of range {expected_domains[col]}', invalid_range_rows)
            accuracy_found = True


# --- Categorical Domains
for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[~df[col].isin(expected_domains[col])].index.tolist()
        if invalid_rows:
            add_issue('Accuracy', col, 'Values not in expected domain list', invalid_rows)
            accuracy_found = True

if not accuracy_found:
    add_issue('Accuracy', 'All relevant columns', 'No accuracy issues found', [])


# --------------------------
# 4 Consistency Checks (Special Characters in Categorical Columns)
special_characters = r'[^a-zA-Z0-9 ._-]'
consistency_found = False

for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[df[col].astype(str).apply(lambda x: bool(re.search(special_characters, x)))].index.tolist()
        if invalid_rows:
            add_issue('Consistency', col, 'Special characters not allowed in categorical fields', invalid_rows)
            consistency_found = True

if not consistency_found:
    add_issue('Consistency', 'All relevant columns', 'No consistency issues found', [])


# --------------------------
# Final Report
dq_report = pd.DataFrame(report)
dq_report = dq_report.sort_values(by='Pillar').reset_index(drop=True)

from IPython.display import display
display(dq_report.style.set_properties(**{'text-align': 'left'})
        .set_caption("Data Quality Report (Accuracy, Completeness, Consistency, Uniqueness)"))


Total Records: 45211
Total Columns: 17


,Pillar,Column Name,Issue description,Row number,Issue Record Count,Percentage
0,Accuracy,All relevant columns,No accuracy issues found,None,0,0.00%
1,Completeness,All Columns,No missing values found,None,0,0.00%
2,Consistency,All relevant columns,No consistency issues found,None,0,0.00%
3,Uniqueness,All Columns,Duplicate records,None,0,0.00%


In [4]:
import pandas as pd
import numpy as np
import re

# --------------------------
# Load Data (Suppress Dtype Warning)
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';', dtype=str)
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';')
total_records = len(df)
total_columns = len(df.columns)
print(f"Total Records: {total_records}")
print(f"Total Columns: {total_columns}")

# --------------------------
# Expected Domains
expected_domains = {
    'age': (18, 95),
    'balance': None,  # No range check on balance
    'day': (1, 31),
    'duration': (0, np.inf),
    'campaign': (1, np.inf),
    'pdays': (-1, 999),
    'previous': (0, 300),

    'job': ["admin.", "unknown", "unemployed", "management", "housemaid",
            "entrepreneur", "student", "blue-collar", "self-employed",
            "retired", "technician", "services"],
    'marital': ["married", "divorced", "single"],
    'education': ["unknown", "secondary", "primary", "tertiary"],
    'default': ["yes", "no"],
    'housing': ["yes", "no"],
    'loan': ["yes", "no"],
    'contact': ["unknown", "telephone", "cellular"],
    'month': ["jan", "feb", "mar", "apr", "may", "jun",
              "jul", "aug", "sep", "oct", "nov", "dec"],
    'poutcome': ["unknown", "other", "failure", "success"],
    'y': ["yes", "no"]
}


categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# --------------------------
# Reporting Utility
report = []

def add_issue(pillar, column, description, rows):
    count = len(rows) if rows else 0
    percentage = (count / total_records) * 100
    if 0 < percentage < 0.01:
        percentage_str = "<0.01%"
    else:
        percentage_str = f"{percentage:.2f}%"
    row_list = rows if rows else None
    report.append({
        'Pillar': pillar,
        'Column Name': column,
        'Issue description': description,
        'Row number': row_list,
        'Issue Record Count': count,
        'Percentage': percentage_str
    })


# --------------------------
# 1 Completeness Check
missing_cols = df.columns[df.isnull().any()].tolist()
if missing_cols:
    rows = df[df[missing_cols].isnull().any(axis=1)].index.tolist()
    add_issue('Completeness', ', '.join(missing_cols), 'Missing values', rows)
else:
   add_issue('Completeness', 'All Columns', 'No missing values found', [])

# --------------------------
# Completeness: "unknown" as missing for categorical fields
# unknown_as_missing_found = False
# for col in categorical_columns:
#     if col in df.columns:
#         unknown_rows = df[df[col] == 'unknown'].index.tolist()
#         if unknown_rows:
#             add_issue('Completeness', col, '"unknown" treated as missing value', unknown_rows)
#             unknown_as_missing_found = True

# if not unknown_as_missing_found:
#     add_issue('Completeness', 'All categorical columns', 'No "unknown" values found', [])



# --------------------------
# 2 Uniqueness Check
duplicates = df[df.duplicated()].index.tolist()
add_issue('Uniqueness', 'All Columns', 'Duplicate records', duplicates)


# --------------------------
# 3 Accuracy Checks

accuracy_found = False

# --- Data Type Validation & Numeric Range Checks (Corrected as requested)
for col in numeric_columns:
    if col in df.columns:
        # Convert to numeric safely
        converted_col = pd.to_numeric(df[col], errors='coerce')

        # Identify rows where conversion failed (non-numeric values)
        non_numeric_rows = df[pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()].index.tolist()
        if non_numeric_rows:
            add_issue('Accuracy', col, 'Non-numeric value found in numeric column', non_numeric_rows)
            accuracy_found = True

        # Skip balance range check as per requirement
        if col == 'balance':
            continue

        # After conversion, check valid ranges
        expected_range = expected_domains[col]
        if expected_range is not None:
            valid_mask = converted_col.between(*expected_range)
            invalid_range_rows = df[~valid_mask & converted_col.notna()].index.tolist()
            if invalid_range_rows:
                add_issue('Accuracy', col, f'Values out of range {expected_range}', invalid_range_rows)
                accuracy_found = True


# --- Categorical Domains
for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[~df[col].isin(expected_domains[col])].index.tolist()
        if invalid_rows:
            add_issue('Accuracy', col, 'Values not in expected domain list', invalid_rows)
            accuracy_found = True

if not accuracy_found:
    add_issue('Accuracy', 'All relevant columns', 'No accuracy issues found', [])


# --------------------------
# 4 Consistency Checks (Special Characters in Categorical Columns)
special_characters = r'[^a-zA-Z0-9 ._-]'
consistency_found = False

for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[df[col].astype(str).apply(lambda x: bool(re.search(special_characters, x)))].index.tolist()
        if invalid_rows:
            add_issue('Consistency', col, 'Special characters not allowed in categorical fields', invalid_rows)
            consistency_found = True

if not consistency_found:
    add_issue('Consistency', 'All relevant columns', 'No consistency issues found', [])


# --------------------------
# Final Report
dq_report = pd.DataFrame(report)
dq_report = dq_report.sort_values(by='Pillar').reset_index(drop=True)

from IPython.display import display
display(dq_report.style.set_properties(**{'text-align': 'left'})
        .set_caption("Data Quality Report (Accuracy, Completeness, Consistency, Uniqueness)"))


Total Records: 45211
Total Columns: 17


,Pillar,Column Name,Issue description,Row number,Issue Record Count,Percentage
0,Accuracy,All relevant columns,No accuracy issues found,None,0,0.00%
1,Completeness,All Columns,No missing values found,None,0,0.00%
2,Consistency,All relevant columns,No consistency issues found,None,0,0.00%
3,Uniqueness,All Columns,Duplicate records,None,0,0.00%


In [5]:
import pandas as pd
import numpy as np
import re
from IPython.display import display

# --------------------------
# Load Data with Original CSV Row Numbers
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';', dtype=str)
df.reset_index(inplace=True)
df['CSV_Row_Number'] = df['index'] + 2  # CSV header is row 1, data starts at 2
df.drop(columns='index', inplace=True)

total_records = len(df)
total_columns = len(df.columns)
print(f"Total Records: {total_records}")
print(f"Total Columns: {total_columns}")

# --------------------------
# Expected Domains
expected_domains = {
    'age': (18, 95),
    'balance': None,
    'day': (1, 31),
    'duration': (0, np.inf),
    'campaign': (1, np.inf),
    'pdays': (-1, 999),
    'previous': (0, 300),
    'job': ["admin.", "unknown", "unemployed", "management", "housemaid",
            "entrepreneur", "student", "blue-collar", "self-employed",
            "retired", "technician", "services"],
    'marital': ["married", "divorced", "single"],
    'education': ["unknown", "secondary", "primary", "tertiary"],
    'default': ["yes", "no"],
    'housing': ["yes", "no"],
    'loan': ["yes", "no"],
    'contact': ["unknown", "telephone", "cellular"],
    'month': ["jan", "feb", "mar", "apr", "may", "jun",
              "jul", "aug", "sep", "oct", "nov", "dec"],
    'poutcome': ["unknown", "other", "failure", "success"],
    'y': ["yes", "no"]
}

categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan',
                       'contact', 'month', 'poutcome', 'y']
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# --------------------------
# Reporting Utility
report = []

def add_issue(pillar, column, description, rows):
    count = len(rows) if rows else 0
    percentage = (count / total_records) * 100
    percentage_str = "<0.01%" if 0 < percentage < 0.01 else f"{percentage:.2f}%"
    row_list = df.loc[rows, 'CSV_Row_Number'].tolist() if rows else None
    report.append({
        'Pillar': pillar,
        'Column Name': column,
        'Issue description': description,
        'Row number': row_list,
        'Issue Record Count': count,
        'Percentage': percentage_str
    })


# --------------------------
# 1 Completeness Check
missing_cols = df.columns[df.isnull().any()].tolist()
if missing_cols:
    rows = df[df[missing_cols].isnull().any(axis=1)].index.tolist()
    add_issue('Completeness', ', '.join(missing_cols), 'Missing values', rows)
else:
    add_issue('Completeness', 'All Columns', 'No missing values found', [])

# --------------------------
# 2 Uniqueness Check
duplicates = df[df.duplicated()].index.tolist()
add_issue('Uniqueness', 'All Columns', 'Duplicate records', duplicates)

# --------------------------
# 3 Accuracy Checks
accuracy_found = False
for col in numeric_columns:
    if col in df.columns:
        converted_col = pd.to_numeric(df[col], errors='coerce')
        non_numeric_rows = df[pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()].index.tolist()
        if non_numeric_rows:
            add_issue('Accuracy', col, 'Non-numeric value found in numeric column', non_numeric_rows)
            accuracy_found = True
        if col == 'balance':
            continue
        expected_range = expected_domains[col]
        if expected_range is not None:
            valid_mask = converted_col.between(*expected_range)
            invalid_range_rows = df[~valid_mask & converted_col.notna()].index.tolist()
            if invalid_range_rows:
                add_issue('Accuracy', col, f'Values out of range {expected_range}', invalid_range_rows)
                accuracy_found = True

for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[~df[col].isin(expected_domains[col])].index.tolist()
        if invalid_rows:
            add_issue('Accuracy', col, 'Values not in expected domain list', invalid_rows)
            accuracy_found = True

if not accuracy_found:
    add_issue('Accuracy', 'All relevant columns', 'No accuracy issues found', [])

# --------------------------
# 4 Consistency Checks (Special Characters in Categorical Columns)
special_characters = r'[^a-zA-Z0-9 ._-]'
consistency_found = False
for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[df[col].astype(str).apply(lambda x: bool(re.search(special_characters, x)))].index.tolist()
        if invalid_rows:
            add_issue('Consistency', col, 'Special characters not allowed in categorical fields', invalid_rows)
            consistency_found = True

if not consistency_found:
    add_issue('Consistency', 'All relevant columns', 'No consistency issues found', [])

# --------------------------
# Final Report
dq_report = pd.DataFrame(report)
dq_report = dq_report.sort_values(by='Pillar').reset_index(drop=True)

display(dq_report.style.set_properties(**{'text-align': 'left'})
        .set_caption("Data Quality Report (Accuracy, Completeness, Consistency, Uniqueness)"))


Total Records: 45211
Total Columns: 18


,Pillar,Column Name,Issue description,Row number,Issue Record Count,Percentage
0,Accuracy,All relevant columns,No accuracy issues found,None,0,0.00%
1,Completeness,All Columns,No missing values found,None,0,0.00%
2,Consistency,All relevant columns,No consistency issues found,None,0,0.00%
3,Uniqueness,All Columns,Duplicate records,None,0,0.00%


In [6]:
import pandas as pd
import numpy as np
import re

# --------------------------
# Load Data (Read only ONCE)
df = pd.read_csv('D:/Projects/bank/bank-full.csv', sep=';', dtype=str)

# Preserve original CSV row numbers aligned to Excel (Header = Row 1)
df.reset_index(inplace=True)
df.rename(columns={'index': 'OriginalRowNumber_ZeroBased'}, inplace=True)
df['CSV_Row_Number'] = df['OriginalRowNumber_ZeroBased'] + 2  # Header + 1-based counting

total_records = len(df)
total_columns = len(df.columns)
print(f"Total Records: {total_records}")
print(f"Total Columns: {total_columns}")
# --------------------------
# Expected Domains
expected_domains = {
    'age': (18, 95),
    'balance': None,
    'day': (1, 31),
    'duration': (0, np.inf),
    'campaign': (1, np.inf),
    'pdays': (-1, 999),
    'previous': (0, 300),
    'job': ["admin.", "unknown", "unemployed", "management", "housemaid",
            "entrepreneur", "student", "blue-collar", "self-employed",
            "retired", "technician", "services"],
    'marital': ["married", "divorced", "single"],
    'education': ["unknown", "secondary", "primary", "tertiary"],
    'default': ["yes", "no"],
    'housing': ["yes", "no"],
    'loan': ["yes", "no"],
    'contact': ["unknown", "telephone", "cellular"],
    'month': ["jan", "feb", "mar", "apr", "may", "jun",
              "jul", "aug", "sep", "oct", "nov", "dec"],
    'poutcome': ["unknown", "other", "failure", "success"],
    'y': ["yes", "no"]
}

categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# --------------------------
# Reporting Utility
report = []
detailed_report = []

def add_issue(pillar, column, description, rows):
    adjusted_rows = df.loc[rows, 'CSV_Row_Number'].tolist() if rows else None
    count = len(rows) if rows else 0
    percentage = (count / total_records) * 100
    percentage_str = "<0.01%" if 0 < percentage < 0.01 else f"{percentage:.2f}%"
    report.append({
        'Pillar': pillar,
        'Column Name': column,
        'Issue description': description,
        'Row number': adjusted_rows,
        'Issue Record Count': count,
        'Percentage': percentage_str
    })

# --------------------------
# Completeness Check
missing_cols = df.columns[df.isnull().any()].tolist()
if missing_cols:
    rows = df[df[missing_cols].isnull().any(axis=1)].index.tolist()
    for r in rows:
        detailed_report.append({
            'Row': df.loc[r, 'CSV_Row_Number'],
            'Pillar': 'Completeness',
            'Column': ', '.join(missing_cols),
            'Actual Value': 'Missing',
            'Expected': 'Non-missing value'
        })
    add_issue('Completeness', ', '.join(missing_cols), 'Missing values', rows)
else:
    add_issue('Completeness', 'All Columns', 'No missing values found', [])

# --------------------------
# Uniqueness Check
duplicates = df[df.duplicated()].index.tolist()
for r in duplicates:
    detailed_report.append({
        'Row': df.loc[r, 'CSV_Row_Number'],
        'Pillar': 'Uniqueness',
        'Column': 'All Columns',
        'Actual Value': 'Duplicate Row',
        'Expected': 'Unique Row'
    })
add_issue('Uniqueness', 'All Columns', 'Duplicate records', duplicates)

# --------------------------
# Accuracy Checks
accuracy_found = False
for col in numeric_columns:
    if col in df.columns:
        converted_col = pd.to_numeric(df[col], errors='coerce')
        non_numeric_rows = df[pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()].index.tolist()
        if non_numeric_rows:
            for r in non_numeric_rows:
                detailed_report.append({
                    'Row': df.loc[r, 'CSV_Row_Number'],
                    'Pillar': 'Accuracy',
                    'Column': col,
                    'Actual Value': df.loc[r, col],
                    'Expected': 'Numeric'
                })
            add_issue('Accuracy', col, 'Non-numeric value found in numeric column', non_numeric_rows)
            accuracy_found = True
        if col == 'balance':
            continue
        expected_range = expected_domains[col]
        if expected_range is not None:
            valid_mask = converted_col.between(*expected_range)
            invalid_range_rows = df[~valid_mask & converted_col.notna()].index.tolist()
            if invalid_range_rows:
                for r in invalid_range_rows:
                    detailed_report.append({
                        'Row': df.loc[r, 'CSV_Row_Number'],
                        'Pillar': 'Accuracy',
                        'Column': col,
                        'Actual Value': df.loc[r, col],
                        'Expected': f"{expected_range[0]} to {expected_range[1]}"
                    })
                add_issue('Accuracy', col, f'Values out of range {expected_range}', invalid_range_rows)
                accuracy_found = True

for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[~df[col].isin(expected_domains[col])].index.tolist()
        if invalid_rows:
            for r in invalid_rows:
                detailed_report.append({
                    'Row': df.loc[r, 'CSV_Row_Number'],
                    'Pillar': 'Accuracy',
                    'Column': col,
                    'Actual Value': df.loc[r, col],
                    'Expected': f"{expected_domains[col]}"
                })
            add_issue('Accuracy', col, 'Values not in expected domain list', invalid_rows)
            accuracy_found = True

if not accuracy_found:
    add_issue('Accuracy', 'All relevant columns', 'No accuracy issues found', [])

# --------------------------
# Consistency Checks (Special Characters in Categorical Columns)
special_characters = r'[^a-zA-Z0-9 ._-]'
consistency_found = False
for col in categorical_columns:
    if col in df.columns:
        invalid_rows = df[df[col].astype(str).apply(lambda x: bool(re.search(special_characters, x)))].index.tolist()
        if invalid_rows:
            for r in invalid_rows:
                detailed_report.append({
                    'Row': df.loc[r, 'CSV_Row_Number'],
                    'Pillar': 'Consistency',
                    'Column': col,
                    'Actual Value': df.loc[r, col],
                    'Expected': 'No special characters allowed'
                })
            add_issue('Consistency', col, 'Special characters not allowed in categorical fields', invalid_rows)
            consistency_found = True

if not consistency_found:
    add_issue('Consistency', 'All relevant columns', 'No consistency issues found', [])

# --------------------------
# --------------------------
# Create DataFrames
dq_report = pd.DataFrame(report).sort_values(by='Pillar').reset_index(drop=True)

detailed_report_df = pd.DataFrame(detailed_report)
if not detailed_report_df.empty:
    detailed_report_df = detailed_report_df.sort_values(by='Row').reset_index(drop=True)

# --------------------------
# Display Summary
from IPython.display import display
display(dq_report.style.set_properties(**{'text-align': 'left'})
        .set_caption("Data Quality Report (Accuracy, Completeness, Consistency, Uniqueness)"))

if not detailed_report_df.empty:
    display(detailed_report_df.style.set_properties(**{'text-align': 'left'})
            .set_caption("Detailed Expected vs Actual Values Report (With Pillars)"))
else:
    print("No issues found.")

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment
from datetime import datetime

# --------------------------
# --------------------------
# Convert lists to strings for Excel
if 'Row number' in dq_report.columns:
    dq_report['Row number'] = dq_report['Row number'].apply(
        lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x
    )

# --------------------------
# Export to Excel
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment
from datetime import datetime

output_file = f"D:/Projects/bank/Data_Quality_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

wb = openpyxl.Workbook()
wb.remove(wb.active)  # remove default sheet

# --------------------------
# Write Summary Report
ws1 = wb.create_sheet(title="Summary_Report")
for r_idx, row in enumerate(dataframe_to_rows(dq_report, index=False, header=True), 1):
    ws1.append(row)
    for c_idx, cell in enumerate(ws1[r_idx], 1):
        if r_idx == 1:
            cell.font = Font(bold=True)
            ws1.auto_filter.ref = ws1.dimensions
        cell.alignment = Alignment(wrap_text=True, vertical='top')

for col in ws1.columns:
    max_length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = max_length + 2

# --------------------------
# Write Detailed Report
ws2 = wb.create_sheet(title="Detailed_Report")
if not detailed_report_df.empty:
    for r_idx, row in enumerate(dataframe_to_rows(detailed_report_df, index=False, header=True), 1):
        ws2.append(row)
        for c_idx, cell in enumerate(ws2[r_idx], 1):
            if r_idx == 1:
                cell.font = Font(bold=True)
                ws2.auto_filter.ref = ws2.dimensions
            cell.alignment = Alignment(wrap_text=True, vertical='top')

    for col in ws2.columns:
        max_length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in col)
        ws2.column_dimensions[col[0].column_letter].width = max_length + 2

# --------------------------
# Save Workbook
wb.save(output_file)
print(f"\n Data Quality Report saved to:\n{output_file}")


Total Records: 45211
Total Columns: 19


,Pillar,Column Name,Issue description,Row number,Issue Record Count,Percentage
0,Accuracy,All relevant columns,No accuracy issues found,None,0,0.00%
1,Completeness,All Columns,No missing values found,None,0,0.00%
2,Consistency,All relevant columns,No consistency issues found,None,0,0.00%
3,Uniqueness,All Columns,Duplicate records,None,0,0.00%


No issues found.

 Data Quality Report saved to:
D:/Projects/bank/Data_Quality_Report_20250721_124112.xlsx
